In [5]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_flow_dynamics(neighbors, x0, lam, beta, dt, T):
    """
    Simulate the dynamics defined by the explicit ODE:

       dx_i/dt = -λ * x_i + β * ∑_{j in neighbors(i)} (x_j - x_i)

    Here, the term β*(x_j - x_i) describes the net influx from the neighbors.
    """
    N = len(x0)
    steps = int(T / dt)
    x = x0.copy()
    history = np.zeros((steps + 1, N))
    history[0] = x
    for t in range(steps):
        dx = np.zeros(N)
        for i in range(N):
            sum_neighbors = sum(x[j] for j in neighbors[i])
            dx[i] = -lam * x[i] + beta * (sum_neighbors - len(neighbors[i]) * x[i])
        x = x + dt * dx
        history[t + 1] = x
    return history

def simulate_energy_dynamics(neighbors, x0, eta, dt, T):
    """
    Simulate dynamics via gradient descent on an energy function.

    Define the energy function:
       E(x) = 1/2 ∑_{(i,j) in edges} (x_i - x_j)^2.

    For an undirected graph, the gradient with respect to x_i is:
       ∂E/∂x_i = ∑_{j in neighbors(i)} (x_i - x_j).

    The gradient descent flow is then given by:
       dx_i/dt = -η * ∂E/∂x_i = -η * ∑_{j in neighbors(i)} (x_i - x_j).
    """
    N = len(x0)
    steps = int(T / dt)
    x = x0.copy()
    history = np.zeros((steps + 1, N))
    history[0] = x
    for t in range(steps):
        dx = np.zeros(N)
        for i in range(N):
            grad = sum(x[i] - x[j] for j in neighbors[i])
            dx[i] = -eta * grad
        x = x + dt * dx
        history[t + 1] = x
    return history

In [9]:
# We define a small undirected graph (symmetric neighbor relations) via a dictionary.
# For each node i, 'neighbors[i]' lists the nodes connected to i.
neighbors = {
    0: [1, 2],
    1: [0, 2, 3],
    2: [0, 1, 3, 4],
    3: [1, 2, 4],
    4: [2, 3]
}

N = 5
x0 = np.random.rand(N)  # initial state for each node

dt = 0.01   # time step
T  = 5.0    # total time
steps = int(T / dt)
time_vals = np.linspace(0, T, steps + 1)

lam = 0.0    # Remove the independent decay term.
beta = 1.0   # Set beta equal to the gradient descent rate (eta).
eta = 1.0    # Gradient descent rate.

history_flow = simulate_flow_dynamics(neighbors, x0, lam, beta, dt, T)
history_energy = simulate_energy_dynamics(neighbors, x0, eta, dt, T)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
for i in range(N):
    plt.plot(time_vals, history_flow[:, i], label=f"Node {i}")
plt.title("Explicit Flow Dynamics")
plt.xlabel("Time")
plt.ylabel("x")
plt.legend()

plt.subplot(1, 2, 2)
for i in range(N):
    plt.plot(time_vals, history_energy[:, i], label=f"Node {i}")
plt.title("Energy-Based Dynamics")
plt.xlabel("Time")
plt.ylabel("x")
plt.legend()

plt.tight_layout()
plt.show()